#  Executive Summary

* There is a strong coefficient correlation between owner_First and owner_Second. We guess that people paying the first-hand car is unlikely to buy the second-hand car, due to their wealth or mysophobia. But in contract, people buying the second or more hand-car would not care so much. So those independent factors (the second or more-hand car) don’t have strong coefficient correlation. 
* 

# Feature Engineering

In [57]:
# import the library that I need
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import math
from sklearn.metrics import mean_squared_error
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [58]:
df = pd.read_csv('cars_india.csv')
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [59]:
df['make']=df.name.str.split().str[0] # split the name by their first name
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,make
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti


In [60]:
df['unite_mileage'] = df.mileage.str.split().str[-1]
df.groupby('unite_mileage').size() # to make sure that the unite of mileage is the same

unite_mileage
km/kg      88
kmpl     7819
dtype: int64

In [61]:
df['n_mileage'] = df.mileage.str.split().str[0]

In [62]:
df['unite_engine'] = df.engine.str.split().str[-1]
df.groupby('unite_engine').size() # to make sure that the units of engine is the same

unite_engine
CC    7907
dtype: int64

In [63]:
df['n_engine'] = df.engine.str.split().str[0]

In [64]:
df['unite_max_power'] = df.max_power.str.split().str[-1]
df.groupby('unite_max_power').size() # to make sure that the units of max_power is the same

unite_max_power
0         6
bhp    7907
dtype: int64

In [65]:
df['n_max_power'] = df.max_power.str.split().str[0]

In [66]:
df['dollars'] = df['selling_price']*0.013 # rupees to US dollars using the current conversion rates.
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,torque,seats,make,unite_mileage,n_mileage,unite_engine,n_engine,unite_max_power,n_max_power,dollars
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,...,190Nm@ 2000rpm,5.0,Maruti,kmpl,23.4,CC,1248,bhp,74,5850.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,...,250Nm@ 1500-2500rpm,5.0,Skoda,kmpl,21.14,CC,1498,bhp,103.52,4810.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,...,"12.7@ 2,700(kgm@ rpm)",5.0,Honda,kmpl,17.7,CC,1497,bhp,78,2054.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,...,22.4 kgm at 1750-2750rpm,5.0,Hyundai,kmpl,23.0,CC,1396,bhp,90,2925.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,...,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti,kmpl,16.1,CC,1298,bhp,88.2,1690.0


In [67]:
df['m_driven'] = df['km_driven'] *1000 # km driven to miles driven
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,seats,make,unite_mileage,n_mileage,unite_engine,n_engine,unite_max_power,n_max_power,dollars,m_driven
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,...,5.0,Maruti,kmpl,23.4,CC,1248,bhp,74,5850.0,145500000
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,...,5.0,Skoda,kmpl,21.14,CC,1498,bhp,103.52,4810.0,120000000
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,...,5.0,Honda,kmpl,17.7,CC,1497,bhp,78,2054.0,140000000
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,...,5.0,Hyundai,kmpl,23.0,CC,1396,bhp,90,2925.0,127000000
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,...,5.0,Maruti,kmpl,16.1,CC,1298,bhp,88.2,1690.0,120000000


In [68]:
df.n_mileage.isna().count() # so there is null row, we can not just directly time a value


8128

In [69]:
def mpg(n):
    if pd.isna(n) == False:
        z = float(n)*2.353
    else:
        z = n
    return z # def a function that would keep the original data if it null, else times 2.353
df['mpg'] = df.apply(lambda row: mpg(row['n_mileage']), axis = 1)
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,make,unite_mileage,n_mileage,unite_engine,n_engine,unite_max_power,n_max_power,dollars,m_driven,mpg
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,...,Maruti,kmpl,23.4,CC,1248,bhp,74,5850.0,145500000,55.06020
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,...,Skoda,kmpl,21.14,CC,1498,bhp,103.52,4810.0,120000000,49.74242
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,...,Honda,kmpl,17.7,CC,1497,bhp,78,2054.0,140000000,41.64810
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,...,Hyundai,kmpl,23.0,CC,1396,bhp,90,2925.0,127000000,54.11900
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,...,Maruti,kmpl,16.1,CC,1298,bhp,88.2,1690.0,120000000,37.88330


In [70]:
df2 = df.copy()
df2 = pd.get_dummies(df2,columns =["transmission", 'owner', 'seller_type', 'fuel'])
df2.head()

,name,year,selling_price,km_driven,mileage,engine,max_power,torque,seats,make,...,owner_Second Owner,owner_Test Drive Car,owner_Third Owner,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol
0,Maruti Swift Dzire VDI,2014,450000,145500,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti,...,0,0,0,0,1,0,0,1,0,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda,...,1,0,0,0,1,0,0,1,0,0
2,Honda City 2017-2020 EXi,2006,158000,140000,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda,...,0,0,1,0,1,0,0,0,0,1
3,Hyundai i20 Sportz Diesel,2010,225000,127000,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai,...,0,0,0,0,1,0,0,1,0,0
4,Maruti Swift VXI BSIII,2007,130000,120000,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti,...,0,0,0,0,1,0,0,0,0,1


In [71]:
df.year.min() # get the lowest year

1983

In [72]:
df2['time'] = df.year - 1983

In [73]:
df2['Honda'] = 0
df2.loc[df2['make'] == 'Honda',['Honda']] = 1

In [74]:
df2.drop(['torque','mileage','selling_price','km_driven','max_power','name','year',\
         'unite_max_power','unite_mileage','unite_engine'],axis=1,inplace = True)
df2.head()

,engine,seats,make,n_mileage,n_engine,n_max_power,dollars,m_driven,mpg,transmission_Automatic,...,owner_Third Owner,seller_type_Dealer,seller_type_Individual,seller_type_Trustmark Dealer,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,time,Honda
0,1248 CC,5.0,Maruti,23.4,1248,74,5850.0,145500000,55.06020,0,...,0,0,1,0,0,1,0,0,31,0
1,1498 CC,5.0,Skoda,21.14,1498,103.52,4810.0,120000000,49.74242,0,...,0,0,1,0,0,1,0,0,31,0
2,1497 CC,5.0,Honda,17.7,1497,78,2054.0,140000000,41.64810,0,...,1,0,1,0,0,0,0,1,23,1
3,1396 CC,5.0,Hyundai,23.0,1396,90,2925.0,127000000,54.11900,0,...,0,0,1,0,0,1,0,0,27,0
4,1298 CC,5.0,Maruti,16.1,1298,88.2,1690.0,120000000,37.88330,0,...,0,0,1,0,0,0,0,1,24,0


In [75]:
df2.isna().count() # check all the columns

engine                          8128
seats                           8128
make                            8128
n_mileage                       8128
n_engine                        8128
n_max_power                     8128
dollars                         8128
m_driven                        8128
mpg                             8128
transmission_Automatic          8128
transmission_Manual             8128
owner_First Owner               8128
owner_Fourth & Above Owner      8128
owner_Second Owner              8128
owner_Test Drive Car            8128
owner_Third Owner               8128
seller_type_Dealer              8128
seller_type_Individual          8128
seller_type_Trustmark Dealer    8128
fuel_CNG                        8128
fuel_Diesel                     8128
fuel_LPG                        8128
fuel_Petrol                     8128
time                            8128
Honda                           8128
dtype: int64

# Regression

In [118]:
X = df2[['Honda','m_driven','fuel_Diesel','fuel_Petrol','fuel_CNG','fuel_LPG','owner_Second Owner',\
         'owner_Third Owner','owner_Fourth & Above Owner']].copy()
Y = df2['dollars'].copy()

In [119]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 2)

In [120]:
X_train = sm.add_constant(X_train) # required if constant expected
est = sm.OLS(y_train,X_train).fit() # fit model
predictions = est.predict() # get predicted values
print(est.summary()) # prints full regression results
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))

                            OLS Regression Results                            
Dep. Variable:                dollars   R-squared:                       0.168
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     102.6
Date:                Tue, 17 Nov 2020   Prob (F-statistic):          3.38e-156
Time:                        15:39:14   Log-Likelihood:                -43108.
No. Observations:                4064   AIC:                         8.623e+04
Df Residuals:                    4055   BIC:                         8.629e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [121]:
X_test = sm.add_constant(X_test) # required if constant expected
est = sm.OLS(y_test,X_test).fit() # fit model
predictions = est.predict() # get predicted values
print(est.summary()) # prints full regression results
print("\nAverage error: {:.2f}.".format(math.sqrt(est.mse_resid)))

                            OLS Regression Results                            
Dep. Variable:                dollars   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.152
Method:                 Least Squares   F-statistic:                     92.05
Date:                Tue, 17 Nov 2020   Prob (F-statistic):          5.89e-141
Time:                        15:39:14   Log-Likelihood:                -42950.
No. Observations:                4064   AIC:                         8.592e+04
Df Residuals:                    4055   BIC:                         8.597e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [122]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = est.model.exog # get model features
vif = pd.DataFrame() # create a dataframe
vif["VIF Factor"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
vif["features"] = X_test.columns
print('VIF: {}'.format(vif))

VIF:    VIF Factor                    features
0    0.000000                       const
1    1.019329                       Honda
2    1.175893                    m_driven
3         inf                 fuel_Diesel
4         inf                 fuel_Petrol
5         inf                    fuel_CNG
6         inf                    fuel_LPG
7    1.102640          owner_Second Owner
8    1.072772           owner_Third Owner
9    1.024553  owner_Fourth & Above Owner


/Users/mac/opt/anaconda3/envs/MISY408/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:1543: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/Users/mac/opt/anaconda3/envs/MISY408/lib/python3.5/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


In [123]:
corr = X_test.corr()
corr.style.background_gradient()

/Users/mac/opt/anaconda3/envs/MISY408/lib/python3.5/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


,const,Honda,m_driven,fuel_Diesel,fuel_Petrol,fuel_CNG,fuel_LPG,owner_Second Owner,owner_Third Owner,owner_Fourth & Above Owner
const,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Honda,nan,1,-0.0329588,-0.11891,0.124746,-0.0211704,-0.0154339,-0.0444384,0.020789,-0.0350085
m_driven,nan,-0.0329588,1,0.258469,-0.260081,-0.00848967,0.020112,0.20174,0.151283,0.0803104
fuel_Diesel,nan,-0.11891,0.258469,1,-0.977429,-0.0934556,-0.0681322,0.0323863,0.00522532,0.00441999
fuel_Petrol,nan,0.124746,-0.260081,-0.977429,1,-0.0777795,-0.0567038,-0.0409135,-0.00680952,-0.00472365
fuel_CNG,nan,-0.0211704,-0.00848967,-0.0934556,-0.0777795,1,-0.00542166,0.0228395,-0.0122361,0.00826943
fuel_LPG,nan,-0.0154339,0.020112,-0.0681322,-0.0567038,-0.00542166,1,0.0359654,0.029221,-0.00896555
owner_Second Owner,nan,-0.0444384,0.20174,0.0323863,-0.0409135,0.0228395,0.0359654,1,-0.158377,-0.0827117
owner_Third Owner,nan,0.020789,0.151283,0.00522532,-0.00680952,-0.0122361,0.029221,-0.158377,1,-0.0389405
owner_Fourth & Above Owner,nan,-0.0350085,0.0803104,0.00441999,-0.00472365,0.00826943,-0.00896555,-0.0827117,-0.0389405,1
